In [ ]:
from glob import glob
import pickle
import csv
import pandas
import numpy as np
from scipy import sparse
import torch
import xml.etree.ElementTree as ET
import os


In [ ]:
paths=[r"D:\Andriod_GCN_andVGAEMalGAN\data\raw\class0\/",r"D:\Andriod_GCN_andVGAEMalGAN\data\raw\class1\/"]
count=0
for l in paths:
    data=os.listdir(l)   
    for md5 in data:
        try:
            intent_list=[]
            premission_list=[]
            root = ET.parse(l+md5+"\AndroidManifest.xml").getroot()
            permissions = root.findall("uses-permission")
            for perm in permissions:
                for att in perm.attrib:
                    premission_list.append(perm.attrib[att])
            for i in root:
                for item in root.findall('application'):
                    for child in item.findall('activity'):
                        for u in child:
                            for k in u.findall('action'):
                                for action in k.attrib:
                                     intent_list.append(k.attrib[action])
                            for k in u.findall('category'):
                                for action in k.attrib:
                                     intent_list.append(k.attrib[action])
            set_intent_list=list(set(intent_list))
            set_permission_list=list(set(premission_list))
            total_list=set_permission_list+set_intent_list
        except Exception as e:
            print(e)
        try:
            if count==0:
                intent_data = open(r'D:\Andriod_GCN_andVGAEMalGAN\permission\class0_permission\/'+md5+".txt",'wb')
            if count==1:
                intent_data = open(r'D:\Andriod_GCN_andVGAEMalGAN\permission\class1_permission\/'+md5+".txt",'wb')    
            pickle.dump(total_list,intent_data)
            intent_data.close()
                
        except Exception as e:
            print(e)
            print("Intent save failed")
    count=count+1

In [ ]:
api_benign_folder=r'D:\Andriod_GCN_andVGAEMalGAN\permission\class0_permission'
api_malware_folder=r'D:\Andriod_GCN_andVGAEMalGAN\permission\class1_permission'
api_benign_paths=glob(f'{api_benign_folder}/*.txt')
api_malware_paths=glob(f'{api_malware_folder}/*.txt')


In [ ]:
api_intents=[]
for i in api_benign_paths:
    with open(i, 'rb') as f:
        prem_intent = pickle.load(f)
        api_intents.append(prem_intent)
print(len(api_intents))

In [ ]:
for i in api_malware_paths:
    with open(i, 'rb') as f:
        prem_intent = pickle.load(f)
        api_intents.append(prem_intent)
print(len(api_intents))

In [ ]:
final_api_intent = [val for sublist in api_intents for val in sublist]
unique_final_api_intent = list(set(final_api_intent))

In [ ]:
unique_final_api_intent = list(filter(None, unique_final_api_intent))
print(len(unique_final_api_intent))

In [ ]:
print(type(unique_final_api_intent))
print(len(unique_final_api_intent))
print(unique_final_api_intent)

In [ ]:
try:
    permission_data = open(r'unique_permission_intent.txt','wb')
    pickle.dump(unique_final_api_intent,permission_data)
    permission_data.close()
except Exception as e:
    print(e)
    print("permission save failed")

In [ ]:

api_Benign_paths=glob(f'{api_benign_folder}/*.txt')
api_malware_paths=glob(f'{api_malware_folder}/*.txt')

In [ ]:
def load_feature(path):
    with open(path, 'rb') as f:
        total_feature = pickle.load(f)
        return total_feature
unique=load_feature(r'unique_permission_intent.txt')

In [ ]:
print(unique)

In [ ]:
paths=[api_Benign_paths,api_malware_paths]
count=0
current=0
unique_final_api_intent_new=["App"]+unique_final_api_intent+["Type"]
print(unique_final_api_intent)
with open(r'permisson.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(unique_final_api_intent_new)
    for path in paths:
        if current==0:
            Type=0
        else:
            Type=1
        for i in path:
            package_name=i.split('\\')[-1].replace(".txt","")
            listofzeros = [0] * len(unique_final_api_intent)
            with open(i, 'rb') as f:
                prem_intent = pickle.load(f)
            prem_intent = list(filter(None, prem_intent))
            for element in prem_intent:
                if element in unique_final_api_intent:
                    index = unique_final_api_intent.index(element)
                    listofzeros[index]=1
                else:
                    print(element+"--"+"not present")
            final_index=[package_name]+listofzeros+[Type]
            write.writerow(final_index)
            count=count+1
            print(count)
        current=current+1

In [ ]:
data_permission = pandas.read_csv(r'permisson.csv')
print(data_permission.shape)
data_permission


In [ ]:
header_list=[]
for i in range(32):
    header_list.append(str("emb")+str(i))
header_list=['App']+header_list+['label']
print(header_list)

In [ ]:
embedding=pandas.read_csv(r'D:\Andriod_GCN_andVGAEMalGAN\GNN\embeddingsage.csv',names=header_list)
embedding

In [ ]:
data_permission = data_permission.drop('Type' , axis='columns')
data_permission

In [ ]:
dfinal = data_permission.merge(embedding, how='inner', left_on='App', right_on='App')
dfinal

In [ ]:
dfinal.to_csv(r'permission_embedding.csv',index=False)